In [7]:
#from google.colab import drive
#drive.mount('/content/drive/')

In [8]:
#import os

#os.chdir('/content/drive/MyDrive/CISC856-Project/')

In [9]:
!rm -rf 'gym-hnef'

In [10]:
!git clone https://github.com/calenirwin/gym-hnef
!pip install -e gym-hnef

Cloning into 'gym-hnef'...
remote: Enumerating objects: 375, done.
remote: Counting objects: 100% (375/375), done.
remote: Compressing objects: 100% (262/262), done.
remote: Total 375 (delta 220), reused 260 (delta 108), pack-reused 0
Receiving objects: 100% (375/375), 492.68 KiB | 30.79 MiB/s, done.
Resolving deltas: 100% (220/220), done.
Obtaining file:///content/gym-hnef
  Found existing installation: gym-hnef 0.0.1
    Can't uninstall 'gym-hnef'. No files were found to uninstall.
  Running setup.py develop for gym-hnef


In [ ]:
import os

def restart_runtime():
  os.kill(os.getpid(), 9)

restart_runtime()

In [2]:
import numpy as np
from matplotlib import pyplot as plt
import gym
from gym_hnef import hnef_game, hnef_vars
from gym_hnef.envs import hnef_env
import model
import mcts
import agent
import config
import funcs
import memory

In [3]:
# 9 * 9 * 9 * 9, size of the action space
action_size = 6561
# initial state of the game
init_state = hnef_game.init_state('historical')

In [4]:
# Create the neural networks
current_NN = model.Residual_CNN(config.REG_CONST, config.LEARNING_RATE, init_state.shape, action_size, config.HIDDEN_CNN_LAYERS)
best_NN = model.Residual_CNN(config.REG_CONST, config.LEARNING_RATE, init_state.shape, action_size, config.HIDDEN_CNN_LAYERS)

In [5]:
# Create the players
current_player = agent.Agent('current_player', current_NN, init_state.shape, action_size)
best_player = agent.Agent('best_player', best_NN, init_state.shape, action_size)

In [6]:
iteration = 0

max_iter = 1e2

mem = memory.Memory(config.MEMORY_SIZE)

version = 0

while iteration < max_iter:
    # Self play
    scores, mem = funcs.play_matches(best_player, best_player, mem=mem)
    print(scores)

    mem.clear_stmemory()

    if len(mem.ltmemory) >= config.MEMORY_SIZE:

        current_player.replay(mem.ltmemory)

        scores, _ = play_matches(best_player, current_player)
        print(scores)

        if scores['current_player'] > scores['best_player'] * config.SCORING_THRESHOLD:
            version += 1
            print(version)
            best_NN.model.set_weights(current_NN.model.get_weights())
    else:
        print('Memory size', str(len(mem.ltmemory)))
    iteration += 1

Iteration number: 0
[((0, 2), (0, 3)), ((0, 4), (1, 4)), ((0, 4), (2, 4)), ((0, 4), (3, 4)), ((0, 4), (0, 3)), ((0, 4), (0, 5)), ((0, 6), (0, 5)), ((1, 2), (2, 2)), ((1, 3), (0, 3)), ((1, 3), (1, 4)), ((1, 5), (0, 5)), ((1, 5), (1, 4)), ((1, 6), (2, 6)), ((2, 0), (3, 0)), ((2, 1), (2, 2)), ((2, 3), (2, 2)), ((2, 3), (2, 4)), ((2, 5), (2, 4)), ((2, 5), (2, 6)), ((2, 7), (2, 6)), ((2, 8), (3, 8)), ((3, 1), (4, 1)), ((3, 1), (3, 0)), ((3, 2), (2, 2)), ((3, 2), (4, 2)), ((3, 3), (4, 3)), ((3, 3), (3, 4)), ((3, 5), (4, 5)), ((3, 5), (3, 4)), ((3, 6), (2, 6)), ((3, 6), (4, 6)), ((3, 7), (4, 7)), ((3, 7), (3, 8)), ((4, 0), (3, 0)), ((4, 0), (5, 0)), ((4, 0), (4, 1)), ((4, 0), (4, 2)), ((4, 0), (4, 3)), ((4, 8), (3, 8)), ((4, 8), (5, 8)), ((4, 8), (4, 7)), ((4, 8), (4, 6)), ((4, 8), (4, 5)), ((5, 1), (4, 1)), ((5, 1), (5, 0)), ((5, 2), (4, 2)), ((5, 2), (6, 2)), ((5, 3), (4, 3)), ((5, 3), (5, 4)), ((5, 5), (4, 5)), ((5, 5), (5, 4)), ((5, 6), (4, 6)), ((5, 6), (6, 6)), ((5, 7), (4, 7)), ((5, 7)

ValueError: ignored